In [1]:
import sagemaker


In [2]:
#Create a sagemaker session to be able to upload data to s3
import boto3
sagemaker_session = sagemaker.Session()

In [3]:
output_bucket = sagemaker.Session().default_bucket()
output_prefix = "sagemaker/pruning_experiment_incremental"
output_bucket_path = f"s3://{output_bucket}"

In [4]:
%%time

import boto3
import sagemaker

from time import gmtime, strftime

TIME_FORMAT = "%Y%m%d_%H%M%S"

role = sagemaker.get_execution_role()
region = boto3.Session().region_name
job_name = f"pruning_experiment_incremental-{strftime(TIME_FORMAT, gmtime())}"
print("Training job", job_name)

Couldn't call 'get_role' to get Role ARN from role name cli_user to get Role path.


ValueError: The current AWS identity is not a role: arn:aws:iam::056057680849:user/cli_user, therefore it cannot be used as a SageMaker execution role

In [24]:
training_image = "056057680849.dkr.ecr.us-east-1.amazonaws.com/chexnet-pruning-experiments:latest"

In [25]:
SECONDS_IN_HOUR = 60*60
HOURS = 120
# estimator = PyTorch(
#     entry_point="main_sagemaker.py",
# #     source_dir=".",
#     instance_type="ml.p3.2xlarge",
#     instance_count=1,
#     volume_size=92,
#     role=role,
#     max_run=HOURS * SECONDS_IN_HOUR,
#     image_uri="763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-inference:1.12.0-gpu-py38-cu116-ubuntu20.04-e3-v1.0",
#
# )
training_config = {
    "TrainingJobName": job_name,
    "AlgorithmSpecification": {
        "TrainingImage": training_image,
        "TrainingInputMode": "File"
    },
    "RoleArn": "arn:aws:iam::056057680849:user/cli_user",
    "OutputDataConfig": {"S3OutputPath": f"{output_bucket_path}/{output_prefix}/tf2-custom"},
    "InputDataConfig": [
        {
            "ChannelName": "dataset",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": "s3://chexnet-dataset-division/",
                }
            },
        },
        {
            "ChannelName": "database",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": "s3://chexnet-dataset-images/",
                }
            },
        },
    ],
    "ResourceConfig": {
        "InstanceType": "ml.p3.2xlarge",
        "InstanceCount": 1,
        "VolumeSizeInGB": 92,
     },
    "StoppingCondition": {
        "MaxRuntimeInSeconds": HOURS * SECONDS_IN_HOUR
    }
}

In [26]:
client = boto3.client("sagemaker", region_name=region)
client.create_training_job(**training_config)

import time

status = client.describe_training_job(TrainingJobName=job_name)["TrainingJobStatus"]
print(status)
while status != "Completed" and status != "Failed":
    time.sleep(60)
    status = client.describe_training_job(TrainingJobName=job_name)["TrainingJobStatus"]
    print(status)

InProgress
InProgress
InProgress
InProgress
InProgress
Completed
